In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document  
import pandas as pd

# 1. 엑셀 파일 로드
xlsx_path = "test용사전엑셀.xlsx"
df = pd.read_excel(xlsx_path)

# 2. 데이터 가공
# 각 row 데이터를 "korean-english" 형식으로 저장
docs = [
    Document(page_content=f'{row["한국어"]}: {row["영어"]}')
    for _, row in df.iterrows()
]

# 3. Embeddings 설정 및 벡터스토어 생성
embeddings = OpenAIEmbeddings()

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

# 4. 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["retrieved_text", "query"],
    template="""
다음은 한국어 텍스트와 관련된 참고 자료입니다:
참고 자료: {retrieved_text}

위 자료를 기반으로, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

translation_chain = LLMChain(
    llm=trans_llm, 
    prompt=translation_prompt
)

C:\Users\201\AppData\Local\Temp\ipykernel_11092\3089089982.py:21: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\201\AppData\Local\Temp\ipykernel_11092\3089089982.py:39: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
C:\Users\201\AppData\Local\Temp\ipykernel_11092\3089089982.py:41: La

In [5]:
query = "산독끼는 바나나와 꼬라니를 좋아해."
retriever = vectorstore.as_retriever()
retrieved_docs = retriever.get_relevant_documents(query)
print("지금 뭐가 있는데?", retrieved_docs )

# 검색 결과를 결합하여 참고 자료로 활용
retrieved_text = " ".join([doc.page_content for doc in retrieved_docs])

result = translation_chain.run({"retrieved_text": retrieved_text, "query": query})
print("번역 결과:", result)

C:\Users\201\AppData\Local\Temp\ipykernel_11092\670826540.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


지금 뭐가 있는데? [Document(id='a1d5c0f0-b80d-40c1-b2cf-16e84497b51e', metadata={}, page_content='산독끼: Zinrabit'), Document(id='9443427a-bb9b-48f8-bb18-b22a7c64113b', metadata={}, page_content='바나나: banana'), Document(id='508d42ca-b1e8-40b9-a2eb-da2fa3fcfb70', metadata={}, page_content='꼬라니: Zingorani'), Document(id='a0eaadce-51dd-45b0-bdcb-8d0d06e71bc8', metadata={}, page_content='고양이: cat')]


C:\Users\201\AppData\Local\Temp\ipykernel_11092\670826540.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = translation_chain.run({"retrieved_text": retrieved_text, "query": query})


번역 결과: Zinrabit likes bananas and Zingorani.
